In [1]:
import pandas as pd
import numpy as np
from Connect import XTSConnect
import json
import re
import datetime
import time

In [ ]:
APIM_KEY = "89da62693cdc2ee3032438"
APIM_SECRET = "Ajsx504@gS"
source = "WEBAPI"

# Initialise
xtm = XTSConnect(APIM_KEY, APIM_SECRET, source)

# Login for authorization token
response_data = xtm.marketdata_login()

print("Login: ", response_data)

In [ ]:
API_KEY = "8957887aa8565c7e42c561"
API_SECRET = "Mrcy250@QM"

XTS_API_BASE_URL = "https://developers.symphonyfintech.in"
source = "WEBAPI"

"""Make XTSConnect object by passing your interactive API appKey, secretKey and source"""
xt = XTSConnect(API_KEY, API_SECRET, source)

"""Using the xt object we created call the interactive login Request"""
response = xt.interactive_login()
print("Login: ", response)

In [ ]:
def ltp_price(exchangeInstrumentID):
     
    instruments = [
        {'exchangeSegment': 2, 'exchangeInstrumentID': exchangeInstrumentID}
    ]

    """Get Quote Request"""
    quote = xtm.get_quote(
        Instruments=instruments,
        xtsMessageCode=1501,
        publishFormat='JSON'
    )

    # Parse the JSON response and extract required data
    quote_data = json.loads(quote['result']['listQuotes'][0])
    ltp_value = quote_data['LastTradedPrice']
    return ltp_value

def quote_time(exchangeInstrumentID):

    instruments = [
        {'exchangeSegment': 2, 'exchangeInstrumentID': exchangeInstrumentID}
    ]

    """Get Quote Request"""
    quote = xtm.get_quote(
        Instruments=instruments,
        xtsMessageCode=1501,
        publishFormat='JSON'
    )

    # Parse the JSON response and extract required data
    quote_data = json.loads(quote['result']['listQuotes'][0])
    epoch = quote_data['LastUpdateTime']
    timezone_offset = 19800  # Offset in seconds (+05:30)
#     ltp = quote_data['LastTradedPrice']
    converted_time = datetime.datetime.fromtimestamp(epoch - timezone_offset).strftime('%Y-%m-%d %H:%M:%S')
    return converted_time
    

def option_inst(ticker,strike):
    """Get Option Symbol Request"""
    option_sym = xtm.get_option_symbol(
       exchangeSegment=2,
       series='OPTIDX',
       symbol='BANKNIFTY',
       expiryDate='28Sep2023',
       optionType=ticker,
       strikePrice=strike)

    return (option_sym['result'][0]['ExchangeInstrumentID'])

In [5]:
def get_average_price_by_instrument_id(exchange_instrument_id):
    
    Orders = xt.get_order_book(clientID)
    for order in Orders['result']:
        if order['ExchangeInstrumentID'] == exchange_instrument_id:
            return order['OrderAverageTradedPrice']
    return None

In [6]:
def ATM_0920():
    for _ in range(100000):

        instruments = [
                    {'exchangeSegment': 1, 'exchangeInstrumentID': 26001}
        ]

        """Get Quote Request"""
        quote = xtm.get_quote(
                Instruments=instruments,
                xtsMessageCode=1512,
                publishFormat='JSON'
        )

        # Parse the JSON response and extract required data\n",
        quote_data = json.loads(quote['result']['listQuotes'][0])
        epoch = quote_data['LastUpdateTime']
        timezone_offset = 19800  # Offset in seconds (+05:30)\n",
        ltp = quote_data['LastTradedPrice']
        converted_time = datetime.datetime.fromtimestamp(epoch - timezone_offset).strftime('%Y-%m-%d %H:%M:%S')
#         print('ltp:', ltp)
#         print('epoch:', epoch)
        print('converted_time:', converted_time)
        position=0
        i=0
        for _ in range(1000000):
            if (converted_time=='2013-09-08 09:20:00') or (converted_time=='2013-09-08 09:20:01') or (converted_time=='2013-09-08 09:20:02')or (converted_time=='2013-09-08 09:20:03'):
                for _ in range(1000000):
#             x=1
#             while x!=0:
             
                    if position == 0:
                        if i==0:

                            instruments = [
                                    {'exchangeSegment': 1, 'exchangeInstrumentID': 26001}
                            ]

                            """Get Quote Request"""
                            quote = xtm.get_quote(
                                Instruments=instruments,
                                xtsMessageCode=1512,
                                publishFormat='JSON'
                            )

                            # Parse the JSON response and extract required data\n",
                            quote_data = json.loads(quote['result']['listQuotes'][0])
                            epoch = quote_data['LastUpdateTime']
                            timezone_offset = 19800  # Offset in seconds (+05:30)\n",
                            ltp = quote_data['LastTradedPrice']

                            converted_time = datetime.datetime.fromtimestamp(epoch - timezone_offset).strftime('%Y-%m-%d %H:%M:%S')
                            print('ltp:', ltp)
                            print('epoch:', epoch)
                            print('converted_time:', converted_time)

                            strike = round(ltp, -2)

                            # Get CE and PE instruments using dictionaries for faster lookups\n
                            ce_inst = option_inst(ticker='CE',strike=strike)
#                             pe_inst = option_inst(ticker='PE',strike=strike)

                            CE_ATM = ltp_price(ce_inst)
#                             PE_ATM = ltp_price(pe_inst)
                            print('CE_ATM',CE_ATM)
#                             print('PE_ATM',PE_ATM)
                    
                            
                            #Buy CE\n",
                            position=1
                            """Place Order Request"""
                            response = xt.place_order(
                               disclosedQuantity=0,
                               exchangeSegment=xt.EXCHANGE_NSEFO,
                               orderType=xt.ORDER_TYPE_MARKET,
                               stopPrice=0,
                               limitPrice=0,
                               orderUniqueIdentifier="454845",
                               exchangeInstrumentID=ce_inst, 
                               orderSide=xt.TRANSACTION_TYPE_BUY,
                               timeInForce=xt.VALIDITY_DAY,
                               productType=xt.PRODUCT_NRML,
                               orderQuantity=15, 
                               clientID=clientID)
                            print("Place Order:", response)
                            OrderID = response['result']['AppOrderID']
                            clientID = response['result']['ClientID']
                            time.sleep(1)
                            Orders = xt.get_order_book(clientID)

                            last_ce_price = None  # Variable to store the last valid 'OrderAverageTradedPrice'\n",

                            for order in Orders['result']:
                                if order['ExchangeInstrumentID'] == ce_inst:
                                    order_average_traded_price = order['OrderAverageTradedPrice'].strip()
                                    if order_average_traded_price:
                                        try:
                                            ce_price = float(order_average_traded_price)
                                            last_ce_price = ce_price
                                            Ordertime = order['OrderGeneratedDateTime']
                                            stoploss = last_ce_price - 15
                                            target = last_ce_price + 30
                                        except ValueError:
                                            print("Error: Unable to convert OrderAverageTradedPrice to float.")
                                    else:
                                            print("Error: OrderAverageTradedPrice is empty.")
                                
                            if last_ce_price is not None:
#                                  ce_sl = last_ce_price
                                ce_value = last_ce_price
                                print('ce_price:',ce_value)
                                sl= stoploss
                                print('stoploss:',sl)
                                tgt = target
                                print('target:',tgt)
                                Starttime=Ordertime 
                                print('Ordertime:',Starttime)
                            
                        elif i == 1:

                            instruments = [
                                    {'exchangeSegment': 1, 'exchangeInstrumentID': 26001}
                            ]

                            """Get Quote Request"""
                            quote = xtm.get_quote(
                                Instruments=instruments,
                                xtsMessageCode=1512,
                                publishFormat='JSON'
                            )

                            # Parse the JSON response and extract required data\n",
                            quote_data = json.loads(quote['result']['listQuotes'][0])
                            epoch = quote_data['LastUpdateTime']
                            timezone_offset = 19800  # Offset in seconds (+05:30)\n",
                            ltp = quote_data['LastTradedPrice']

                            converted_time = datetime.datetime.fromtimestamp(epoch - timezone_offset).strftime('%Y-%m-%d %H:%M:%S')
                            print('ltp:', ltp)
                            print('epoch:', epoch)
                            print('converted_time:', converted_time)

                            strike = round(ltp, -2)

                            # Get CE and PE instruments using dictionaries for faster lookups\n
#                             ce_inst = option_inst(ticker='CE',strike=strike)
                            pe_inst = option_inst(ticker='PE',strike=strike)

#                             CE_ATM = ltp_price(ce_inst)
                            PE_ATM = ltp_price(pe_inst)
#                             print('CE_ATM',CE_ATM)
                            print('PE_ATM',PE_ATM)
 
                            API_KEY = "8957887aa8565c7e42c561"
                            API_SECRET = "Mrcy250@QM"
                            source = "WEBAPI"
                    
                            """Make XTSConnect object by passing your interactive API appKey, secretKey and source"""
                            xt = XTSConnect(API_KEY, API_SECRET, source)

                            """Using the xt object we created call the interactive login Request"""
                            response = xt.interactive_login()
                            #print(\"Login: \", response)\n",
                            clientID= 'J915'
                            
                            position=-1
                            response = xt.place_order(
                                    disclosedQuantity=0,
                                    exchangeSegment=xt.EXCHANGE_NSEFO,
                                    orderType=xt.ORDER_TYPE_MARKET,
                                    stopPrice=0,
                                    limitPrice=0,
                                    orderUniqueIdentifier="454845",
                                    exchangeInstrumentID=pe_inst,
                                    orderSide=xt.TRANSACTION_TYPE_BUY,
                                    timeInForce=xt.VALIDITY_DAY,
                                    productType=xt.PRODUCT_NRML,
                                    orderQuantity=15, 
                                    clientID=clientID)
                            print("Place Order: ", response)
                            OrderID = response['result']['AppOrderID']
                            clientID = response['result']['ClientID']
                            time.sleep(1)
                            Orders = xt.get_order_book(clientID)

                            last_pe_price = None  # Variable to store the last valid 'OrderAverageTradedPrice'\n",

                            for order in Orders['result']:
                                if order['ExchangeInstrumentID'] == pe_inst:
                                    order_average_traded_price = order['OrderAverageTradedPrice'].strip()
                                    if order_average_traded_price:
                                        try:
                                            pe_price = float(order_average_traded_price)
                                            last_pe_price = pe_price
                                            Ordertime = order['OrderGeneratedDateTime']
                                            stoploss = last_pe_price - 15
                                            target = last_pe_price + 30
                                        except ValueError:
                                            print("Error: Unable to convert OrderAverageTradedPrice to float.")
                                    else:
                                            print("Error: OrderAverageTradedPrice is empty.")
                                
                            if last_pe_price is not None:
#                                  ce_sl = last_ce_price
                                pe_value = last_pe_price
                                print('pe_price:',pe_value)
                                sl= stoploss
                                print('stoploss:',sl)
                                tgt = target
                                print('target:',tgt)
                                Starttime=Ordertime 
                                print('Ordertime:',Starttime)

                    
                    elif position==1:
                        if ltp_price(ce_inst)<=sl:
                            position=0
                            i=1
                            response = xt.place_order(
                                disclosedQuantity=0,
                                exchangeSegment=xt.EXCHANGE_NSEFO,
                                orderType=xt.ORDER_TYPE_MARKET,
                                stopPrice=0,
                                limitPrice=0,
                                orderUniqueIdentifier="454845",
                                exchangeInstrumentID=ce_inst,
                                orderSide=xt.TRANSACTION_TYPE_SELL,
                                timeInForce=xt.VALIDITY_DAY,
                                productType=xt.PRODUCT_NRML,
                                orderQuantity=15,
                                clientID=clientID)
                            print("Place Order:", response)

                            time.sleep(1)
                            Orders = xt.get_order_book(clientID)

                            Exit_ce_price = None  # Variable to store the last valid 'OrderAverageTradedPrice'\n",
                            for order in Orders['result']:
                                if order['ExchangeInstrumentID'] == ce_inst:
                                    order_average_traded_price = order['OrderAverageTradedPrice'].strip()
                                    if order_average_traded_price:
                                        try:
                                            ce_price_exit = float(order_average_traded_price)
                                            Exit_ce_price = ce_price_exit
                                            Ordertime_Exit = order['OrderGeneratedDateTime']
                                        except ValueError:
                                            print("Error: Unable to convert OrderAverageTradedPrice to float.")
                                    else:
                                        print("Error: OrderAverageTradedPrice is empty.")

                        # Now last_ce_price contains the last valid 'OrderAverageTradedPrice'\n",
                            if Exit_ce_price is not None:
                                    ce_exit_value = Exit_ce_price
                                    print('ce_price_Exit:',ce_exit_value)
                                    Exittime=Ordertime_Exit 
                                    print('Exittime:',Exittime)
                                
                                
                        elif ltp_price(ce_inst)>=tgt:
                                    position=0
                                    i=0
                                    response = xt.place_order(
                                        disclosedQuantity=0,
                                        exchangeSegment=xt.EXCHANGE_NSEFO,
                                        orderType=xt.ORDER_TYPE_MARKET,
                                        stopPrice=0,
                                        limitPrice=0,
                                        orderUniqueIdentifier="454845",
                                        exchangeInstrumentID=ce_inst, 
                                        orderSide=xt.TRANSACTION_TYPE_SELL,
                                        timeInForce=xt.VALIDITY_DAY,
                                        productType=xt.PRODUCT_NRML,
                                        orderQuantity=15,
                                        clientID=clientID)
                                    print("Place Order: ", response)

                                    time.sleep(1)
                                    Orders = xt.get_order_book(clientID)

                                    Exit_ce_price = None  # Variable to store the last valid 'OrderAverageTradedPrice'\n",
                                    for order in Orders['result']:
                                        if order['ExchangeInstrumentID'] == ce_inst:
                                            order_average_traded_price = order['OrderAverageTradedPrice'].strip()
                                            if order_average_traded_price:
                                                try:
                                                    ce_price_exit = float(order_average_traded_price)
                                                    Exit_ce_price = ce_price_exit
                                                    Ordertime_Exit = order['OrderGeneratedDateTime']
                                                except ValueError:
                                                    print("Error: Unable to convert OrderAverageTradedPrice to float.")
                                            else:
                                                print("Error: OrderAverageTradedPrice is empty.")

                        # Now last_ce_price contains the last valid 'OrderAverageTradedPrice'\n",
                                    if Exit_ce_price is not None:
                                        ce_exit_value = Exit_ce_price
                                        print('ce_price_Exit:',ce_exit_value)
                                        Exittime=Ordertime_Exit 
                                        print('Exittime:',Exittime)



                    elif position==-1:
                        if ltp_price(pe_inst)<=sl:
                            position=0
                            i=0
                            response = xt.place_order(
                                disclosedQuantity=0,
                                exchangeSegment=xt.EXCHANGE_NSEFO,
                                orderType=xt.ORDER_TYPE_MARKET,
                                stopPrice=0,
                                limitPrice=0,
                                orderUniqueIdentifier="454845",
                                exchangeInstrumentID=pe_inst,
                                orderSide=xt.TRANSACTION_TYPE_SELL,
                                timeInForce=xt.VALIDITY_DAY,
                                productType=xt.PRODUCT_NRML,
                                orderQuantity=15,
                                clientID=clientID)
                            print("Place Order: ", response)

                            time.sleep(1)
                            Orders = xt.get_order_book(clientID)

                            Exit_ce_price = None  # Variable to store the last valid 'OrderAverageTradedPrice'\n",
                            for order in Orders['result']:
                                if order['ExchangeInstrumentID'] == pe_inst:
                                    order_average_traded_price = order['OrderAverageTradedPrice'].strip()
                                    if order_average_traded_price:
                                        try:
                                            pe_price_exit = float(order_average_traded_price)
                                            Exit_pe_price = pe_price_exit
                                            Ordertime_Exit = order['OrderGeneratedDateTime']
                                        except ValueError:
                                            print("Error: Unable to convert OrderAverageTradedPrice to float.")
                                    else:
                                        print("Error: OrderAverageTradedPrice is empty.")

                        # Now last_ce_price contains the last valid 'OrderAverageTradedPrice'\n",
                            if Exit_pe_price is not None:
                                    pe_exit_value = Exit_pe_price
                                    print('pe_price_Exit:',pe_exit_value)
                                    Exittime=Ordertime_Exit
                                    print('Exittime:',Exittime)



                        elif ltp_price(pe_inst)>=tgt:
                                    position=0
                                    i=1
                                    response = xt.place_order(
                                        disclosedQuantity=0,
                                        exchangeSegment=xt.EXCHANGE_NSEFO,
                                        orderType=xt.ORDER_TYPE_MARKET,
                                        stopPrice=0,
                                        limitPrice=0,
                                        orderUniqueIdentifier="454845",
                                        exchangeInstrumentID=pe_inst,
                                        orderSide=xt.TRANSACTION_TYPE_SELL,
                                        timeInForce=xt.VALIDITY_DAY,
                                        productType=xt.PRODUCT_NRML,
                                        orderQuantity=15,
                                        clientID=clientID)
                                    print("Place Order:", response)
    
                                    time.sleep(1)
                                    Orders = xt.get_order_book(clientID)
    
                                    Exit_ce_price = None  # Variable to store the last valid 'OrderAverageTradedPrice'\n",
                                    for order in Orders['result']:
                                        if order['ExchangeInstrumentID'] == pe_inst:
                                            order_average_traded_price = order['OrderAverageTradedPrice'].strip()
                                            if order_average_traded_price:
                                                try:
                                                    pe_price_exit = float(order_average_traded_price)
                                                    Exit_pe_price = pe_price_exit
                                                    Ordertime_Exit = order['OrderGeneratedDateTime']
                                                except ValueError:
                                                    print("Error: Unable to convert OrderAverageTradedPrice to float.")
                                            else:
                                                print("Error: OrderAverageTradedPrice is empty.")
                        # Now last_ce_price contains the last valid 'OrderAverageTradedPrice'
                                    if Exit_pe_price is not None:
                                        pe_exit_value = Exit_pe_price
                                        print('pe_price_Exit:',pe_exit_value)
                                        Exittime=Ordertime_Exit
                                        print('Exittime:',Exittime)                       
                                        
            else:
                continue

In [7]:
ATM_0920()

converted_time: 2013-09-08 09:15:01
converted_time: 2013-09-08 09:15:01
converted_time: 2013-09-08 09:15:01
converted_time: 2013-09-08 09:15:01
converted_time: 2013-09-08 09:15:01
converted_time: 2013-09-08 09:15:01
converted_time: 2013-09-08 09:15:01
converted_time: 2013-09-08 09:15:01
converted_time: 2013-09-08 09:15:01
converted_time: 2013-09-08 09:15:01
converted_time: 2013-09-08 09:15:01
converted_time: 2013-09-08 09:15:03
converted_time: 2013-09-08 09:15:03
converted_time: 2013-09-08 09:15:03
converted_time: 2013-09-08 09:15:03
converted_time: 2013-09-08 09:15:03
converted_time: 2013-09-08 09:15:03
converted_time: 2013-09-08 09:15:03
converted_time: 2013-09-08 09:15:03
converted_time: 2013-09-08 09:15:03
converted_time: 2013-09-08 09:15:03
converted_time: 2013-09-08 09:15:03
converted_time: 2013-09-08 09:15:05
converted_time: 2013-09-08 09:15:05
converted_time: 2013-09-08 09:15:05
converted_time: 2013-09-08 09:15:05
converted_time: 2013-09-08 09:15:05
converted_time: 2013-09-08 0

converted_time: 2013-09-08 09:15:43
converted_time: 2013-09-08 09:15:43
converted_time: 2013-09-08 09:15:43
converted_time: 2013-09-08 09:15:43
converted_time: 2013-09-08 09:15:43
converted_time: 2013-09-08 09:15:43
converted_time: 2013-09-08 09:15:43
converted_time: 2013-09-08 09:15:43
converted_time: 2013-09-08 09:15:43
converted_time: 2013-09-08 09:15:45
converted_time: 2013-09-08 09:15:45
converted_time: 2013-09-08 09:15:45
converted_time: 2013-09-08 09:15:45
converted_time: 2013-09-08 09:15:45
converted_time: 2013-09-08 09:15:45
converted_time: 2013-09-08 09:15:45
converted_time: 2013-09-08 09:15:45
converted_time: 2013-09-08 09:15:45
converted_time: 2013-09-08 09:15:45
converted_time: 2013-09-08 09:15:45
converted_time: 2013-09-08 09:15:47
converted_time: 2013-09-08 09:15:47
converted_time: 2013-09-08 09:15:47
converted_time: 2013-09-08 09:15:47
converted_time: 2013-09-08 09:15:47
converted_time: 2013-09-08 09:15:47
converted_time: 2013-09-08 09:15:47
converted_time: 2013-09-08 0

converted_time: 2013-09-08 09:16:25
converted_time: 2013-09-08 09:16:25
converted_time: 2013-09-08 09:16:25
converted_time: 2013-09-08 09:16:25
converted_time: 2013-09-08 09:16:25
converted_time: 2013-09-08 09:16:25
converted_time: 2013-09-08 09:16:25
converted_time: 2013-09-08 09:16:27
converted_time: 2013-09-08 09:16:27
converted_time: 2013-09-08 09:16:27
converted_time: 2013-09-08 09:16:27
converted_time: 2013-09-08 09:16:27
converted_time: 2013-09-08 09:16:27
converted_time: 2013-09-08 09:16:27
converted_time: 2013-09-08 09:16:27
converted_time: 2013-09-08 09:16:27
converted_time: 2013-09-08 09:16:27
converted_time: 2013-09-08 09:16:27
converted_time: 2013-09-08 09:16:29
converted_time: 2013-09-08 09:16:29
converted_time: 2013-09-08 09:16:29
converted_time: 2013-09-08 09:16:29
converted_time: 2013-09-08 09:16:29
converted_time: 2013-09-08 09:16:29
converted_time: 2013-09-08 09:16:29
converted_time: 2013-09-08 09:16:29
converted_time: 2013-09-08 09:16:29
converted_time: 2013-09-08 0

converted_time: 2013-09-08 09:17:08
converted_time: 2013-09-08 09:17:08
converted_time: 2013-09-08 09:17:10
converted_time: 2013-09-08 09:17:10
converted_time: 2013-09-08 09:17:10
converted_time: 2013-09-08 09:17:10
converted_time: 2013-09-08 09:17:10
converted_time: 2013-09-08 09:17:10
converted_time: 2013-09-08 09:17:10
converted_time: 2013-09-08 09:17:10
converted_time: 2013-09-08 09:17:10
converted_time: 2013-09-08 09:17:10
converted_time: 2013-09-08 09:17:10
converted_time: 2013-09-08 09:17:11
converted_time: 2013-09-08 09:17:11
converted_time: 2013-09-08 09:17:11
converted_time: 2013-09-08 09:17:11
converted_time: 2013-09-08 09:17:11
converted_time: 2013-09-08 09:17:11
converted_time: 2013-09-08 09:17:11
converted_time: 2013-09-08 09:17:11
converted_time: 2013-09-08 09:17:11
converted_time: 2013-09-08 09:17:11
converted_time: 2013-09-08 09:17:11
converted_time: 2013-09-08 09:17:14
converted_time: 2013-09-08 09:17:14
converted_time: 2013-09-08 09:17:14
converted_time: 2013-09-08 0

converted_time: 2013-09-08 09:17:52
converted_time: 2013-09-08 09:17:52
converted_time: 2013-09-08 09:17:52
converted_time: 2013-09-08 09:17:52
converted_time: 2013-09-08 09:17:52
converted_time: 2013-09-08 09:17:52
converted_time: 2013-09-08 09:17:54
converted_time: 2013-09-08 09:17:54
converted_time: 2013-09-08 09:17:54
converted_time: 2013-09-08 09:17:54
converted_time: 2013-09-08 09:17:54
converted_time: 2013-09-08 09:17:54
converted_time: 2013-09-08 09:17:54
converted_time: 2013-09-08 09:17:54
converted_time: 2013-09-08 09:17:54
converted_time: 2013-09-08 09:17:54
converted_time: 2013-09-08 09:17:56
converted_time: 2013-09-08 09:17:56
converted_time: 2013-09-08 09:17:56
converted_time: 2013-09-08 09:17:56
converted_time: 2013-09-08 09:17:56
converted_time: 2013-09-08 09:17:56
converted_time: 2013-09-08 09:17:56
converted_time: 2013-09-08 09:17:56
converted_time: 2013-09-08 09:17:56
converted_time: 2013-09-08 09:17:56
converted_time: 2013-09-08 09:17:58
converted_time: 2013-09-08 0

...

Place Order: {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200045150, 'OrderUniqueIdentifier': '454845', 'ClientID': 'J915'}}
ce_price: 222.65
stoploss: 207.65
target: 252.65
Ordertime: 08-09-2023 10:38:21
Place Order: {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200045204, 'OrderUniqueIdentifier': '454845', 'ClientID': 'J915'}}
ce_price_Exit: 208.1
Exittime: 08-09-2023 10:39:22
ltp: 44845.6
epoch: 1378636762
converted_time: 2013-09-08 10:39:22
PE_ATM 220.5
Place Order:  {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200045205, 'OrderUniqueIdentifier': '454845', 'ClientID': 'J915'}}
pe_price: 219.95
stoploss: 204.95
target: 249.95
Ordertime: 08-09-2023 10:39:23
Place Order:  {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200045328, 'OrderUniqueIdentifier': '454845', 

pe_price: 212.3
stoploss: 197.3
target: 242.3
Ordertime: 08-09-2023 11:45:52
Place Order:  {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200053525, 'OrderUniqueIdentifier': '454845', 'ClientID': 'J915'}}
pe_price_Exit: 192.3
Exittime: 08-09-2023 12:38:56
ltp: 45151.9
epoch: 1378643935
converted_time: 2013-09-08 12:38:55
CE_ATM 250
Place Order: {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200053532, 'OrderUniqueIdentifier': '454845', 'ClientID': 'J915'}}
ce_price: 254.4
stoploss: 239.4
target: 284.4
Ordertime: 08-09-2023 12:38:57
Place Order: {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200054267, 'OrderUniqueIdentifier': '454845', 'ClientID': 'J915'}}
ce_price_Exit: 237.0
Exittime: 08-09-2023 12:45:57
ltp: 45147.2
epoch: 1378644357
converted_time: 2013-09-08 12:45:57
PE_ATM 210.25
Place Order:  {'type': 'success', 'cod

Error: OrderAverageTradedPrice is empty.
ce_price_Exit: 249.45
Exittime: 08-09-2023 14:14:53
ltp: 45264.9
epoch: 1378649768
converted_time: 2013-09-08 14:16:08
CE_ATM 223.4
Place Order: {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200058452, 'OrderUniqueIdentifier': '454845', 'ClientID': 'J915'}}
Error: OrderAverageTradedPrice is empty.
Place Order: {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200058519, 'OrderUniqueIdentifier': '454845', 'ClientID': 'J915'}}
ce_price_Exit: 225.0
Exittime: 08-09-2023 14:16:12
ltp: 45273.65
epoch: 1378649772
converted_time: 2013-09-08 14:16:12
PE_ATM 253.45
Place Order:  {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200058541, 'OrderUniqueIdentifier': '454845', 'ClientID': 'J915'}}
pe_price: 254.0
stoploss: 239.0
target: 284.0
Ordertime: 08-09-2023 14:16:13
Place Order:  {'type': 'succe

PE_ATM 211.65
Place Order:  {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200062898, 'OrderUniqueIdentifier': '454845', 'ClientID': 'J915'}}
pe_price: 211.95
stoploss: 196.95
target: 241.95
Ordertime: 08-09-2023 14:50:24
Place Order:  {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200062915, 'OrderUniqueIdentifier': '454845', 'ClientID': 'J915'}}
pe_price_Exit: 195.25
Exittime: 08-09-2023 14:51:23
ltp: 45161
epoch: 1378651883
converted_time: 2013-09-08 14:51:23
CE_ATM 221.8
Place Order: {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200062916, 'OrderUniqueIdentifier': '454845', 'ClientID': 'J915'}}
ce_price: 221.25
stoploss: 206.25
target: 251.25
Ordertime: 08-09-2023 14:51:25
Place Order: {'type': 'success', 'code': 's-orders-0001', 'description': 'Request sent', 'result': {'AppOrderID': 1200062973, 'OrderUniqueIdentifier

KeyboardInterrupt: 